## Project Utilities

In [ ]:
# Parameters
RAW_DIR       = "data/raw/"
INTERIM_DIR   = "data/interim/"
PROCESSED_DIR = "data/processed/"
MODELS_DIR    = "models/"
FIGURES_DIR   = "figures/"
TRAIN_END     = "2024-09-30"
SEED          = 42

# 1. Imports & environment check
import os, sys, logging
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX
import lightgbm as lgb
from sklearn.metrics import (
    mean_absolute_error,
    mean_absolute_percentage_error,
    mean_squared_error
)

from sklearn.model_selection import TimeSeriesSplit



# Configure plotting & logging
try:
    plt.style.use("seaborn")  # fallback to core seaborn style
except Exception:
    pass
sns.set_context("talk")
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")

# 2. Directory setup
for d in [INTERIM_DIR, PROCESSED_DIR, MODELS_DIR, FIGURES_DIR]:
    os.makedirs(d, exist_ok=True)
logging.info(f"Directories ready: {INTERIM_DIR}, {PROCESSED_DIR}, {MODELS_DIR}, {FIGURES_DIR}")

# 3. Helper functions (also extract these into a file utils.py in your project root)
def load_csv(path: str) -> pd.DataFrame:
    df = pd.read_csv(path, parse_dates=True, infer_datetime_format=True)
    logging.info(f"Loaded {path} with {len(df):,} rows.")
    return df


def save_df(df: pd.DataFrame, path: str):
    df.to_csv(path, index=False)
    logging.info(f"Saved DataFrame to {path} with {df.shape[0]:,} rows.")


def detect_anomalies(df: pd.Series, method: str='iqr', k: float=1.5) -> pd.Series:
    if method=='iqr':
        q1, q3 = df.quantile([0.25,0.75])
        iqr = q3 - q1
        return (df < q1 - k*iqr) | (df > q3 + k*iqr)
        # return (df > q3 + k*iqr)
    elif method=='z':
        return (df - df.mean()).abs()/df.std() > k
    else:
        raise ValueError("method must be 'iqr' or 'z'")


def evaluate_forecast(y_true: pd.Series, y_pred: pd.Series) -> dict:
    """Aligns true/predicted, drops NaNs, masks near-zero y_true, and computes MAE, MAPE, RMSE."""
    # 1. Align and drop any NaNs
    y_pred = y_pred.reindex(y_true.index)
    df_cmp = pd.concat([
        y_true.rename('y_true'),
        y_pred.rename('y_pred')
    ], axis=1).dropna()
    if df_cmp.empty:
        return {'MAE': np.nan, 'MAPE': np.nan, 'RMSE': np.nan}

    # 2. Mask out near-zero true values
    mask = df_cmp['y_true'].abs() > MAPE_THRESHOLD
    df_masked = df_cmp.loc[mask]
    if df_masked.empty:
        return {'MAE': np.nan, 'MAPE': np.nan, 'RMSE': np.nan}

    yt = df_masked['y_true']
    yp = df_masked['y_pred']

    # 3. Compute metrics
    return {
        'MAE':  mean_absolute_error(yt, yp),
        'MAPE': mean_absolute_percentage_error(yt, yp),
        'RMSE': np.sqrt(mean_squared_error(yt, yp))
    }

def create_lag_features(df: pd.DataFrame, target: str, lags: list[int]):
    for lag in lags:
        df[f"{target}_lag{lag}"] = df[target].shift(lag)
    return df

2025-04-24 14:00:39,999 INFO Directories ready: data/interim/, data/processed/, models/, figures/


## Data ingestion and preprocessing

In [ ]:
from project_utils import save_df

2025-04-24 14:00:40,021 INFO Directories ready: data/interim/, data/processed/, models/, figures/


### Data Loading

In [ ]:
# Parameters
POS_PATH    = '../modeling_ivy/processed_data_20250416/transaction_master_biggs_cleaned.csv'

# Load transaction-level data (cleaned)
pos      = pd.read_csv(POS_PATH, parse_dates=['date'])
pos = pos.rename(columns={'ite_desc_std': 'ite_desc_standardized'})

### Aggregate to daily

In [ ]:
## Aggregate to daily branch×item

pos_daily = (
pos
.groupby(['branch','ite_desc_standardized','date'], as_index=False)
['quantity'].sum()
)

### [CHECK] Remove after running the notebook end-to-end

In [ ]:
## Filter to 2023–2024 and select top 10 items per branch

df_23_24 = pos_daily[(pos_daily['date'].dt.year >= 2023) & (pos_daily['date'].dt.year <= 2024)]
top_items = (
df_23_24
.groupby(['branch','ite_desc_standardized'])['quantity']
.sum()
.reset_index()
.sort_values(['branch','quantity'], ascending=[True, False])
.groupby('branch')
.head(10)
)
pos_daily = pos_daily.merge(
top_items[['branch','ite_desc_standardized']],
on=['branch','ite_desc_standardized'],
how='inner'
)

### Save daily sales

In [ ]:
save_df(pos_daily, 'data/interim/daily_branch_item.csv')

2025-04-24 14:01:28,058 INFO Saved DataFrame to data/interim/daily_branch_item.csv with 15,985 rows.


## Anomaly Detection

### Input: Daily Sales

In [ ]:
# Parameters
INPUT_CSV = 'data/interim/daily_branch_item.csv'

# Load data
df = pd.read_csv(INPUT_CSV, parse_dates=['date'])

### [CHECK] Restrict data series for anomaly detection and treatment

In [ ]:
# Restrict to 2023–2024 for anomaly detection and treatment
df = df[(df['date'].dt.year >= 2023) & (df['date'].dt.year <= 2024)]

In [ ]:
# Extract time-based features
df['dayofweek'] = df['date'].dt.dayofweek
df['month'] = df['date'].dt.month
df['week'] = df['date'].dt.isocalendar().week

### Identify anomalies

In [ ]:
# Flag anomalies
df['anomaly'] = (
    df
    .groupby(['branch', 'ite_desc_standardized'])['quantity']
    .transform(lambda x: detect_anomalies(x, method='iqr', k=1.5))
)

### Compute seasonal median values

In [ ]:
# Seasonal typical (median) values
seasonal = (
    df.groupby(['branch', 'ite_desc_standardized', 'month'])['quantity']
      .median()
      .rename('typical')
      .reset_index()
)


### Merge the seasonal typicals and come up with `quantity_cap`

In [ ]:
# Merge seasonal typicals back
df = df.merge(seasonal, on=['branch', 'ite_desc_standardized', 'month'], how='left')

# Replace outliers with seasonal typicals
df['quantity_cap'] = np.where(df['anomaly'], df['typical'], df['quantity'])

### Save capped daily sales

In [ ]:
# Save output
df.to_csv('data/processed/flagged.csv', index=False)  # Full data with anomaly flags and capped values
df[['branch', 'ite_desc_standardized', 'date', 'quantity_cap']].to_csv('data/processed/capped.csv', index=False)  # Cleaned dataset

### ARIMA Modeling

In [3]:
# === Simplified Happy Path Forecasting Pipeline: ARIMA on All Series (One Month Ahead) ===
"""
This script runs an end-to-end ARIMA forecasting pipeline
for every branch & item series in the capped.csv file,
forecasting one calendar month ahead from TRAIN_END.
"""
import os
import logging
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

# Configuration
class Config:
    ARIMA_ORDER = (1, 1, 1)            # <-- Customize (p, d, q)
    TRAIN_END = '2024-03-31'           # last date of training data
    DATA_PATH = os.path.join('data', 'processed', 'capped.csv')
    OUTPUT_DIR = os.path.join('results')
    DATE_COL = 'date'
    BRANCH_COL = 'branch'
    ITEM_COL = 'ite_desc_standardized'
    QUANT_COL = 'quantity_cap'

# Setup
os.makedirs(Config.OUTPUT_DIR, exist_ok=True)
logging.basicConfig(level=logging.INFO)

# 1. Load data
df = pd.read_csv(Config.DATA_PATH, parse_dates=[Config.DATE_COL])

# 2. Determine forecast horizon (one month ahead)
train_end_dt = pd.to_datetime(Config.TRAIN_END)
next_month_end = train_end_dt + pd.offsets.MonthEnd(1)
horizon = (next_month_end - train_end_dt).days
logging.info(f"Forecast horizon: {horizon} days until {next_month_end.date()}")

# Prepare storage
all_results = []
evaluations = []

# 3. Loop over each branch-item combination
series_keys = df[[Config.BRANCH_COL, Config.ITEM_COL]].drop_duplicates()
for _, row in series_keys.iterrows():
    branch = row[Config.BRANCH_COL]
    item = row[Config.ITEM_COL]
    subset = df[(df[Config.BRANCH_COL] == branch) &
                (df[Config.ITEM_COL] == item)].copy()
    if subset.empty:
        continue

    # Sort and index by date
    subset = subset.sort_values(Config.DATE_COL).set_index(Config.DATE_COL)
    train_series = subset.loc[:Config.TRAIN_END, Config.QUANT_COL]
    test_series = subset.loc[Config.TRAIN_END:, Config.QUANT_COL].iloc[:horizon]

    if train_series.empty:
        logging.warning(f"Skipping {branch} | {item}: no training data.")
        continue

    # Fit ARIMA
    try:
        model = ARIMA(train_series, order=Config.ARIMA_ORDER)
        fit = model.fit()
    except Exception as e:
        logging.warning(f"ARIMA fit failed for {branch} | {item}: {e}")
        continue

    # Forecast
    dates_fc = pd.date_range(
        start=train_end_dt + pd.Timedelta(days=1),
        periods=horizon,
        freq='D'
    )
    fc_values = fit.forecast(steps=horizon)
    fc_series = pd.Series(fc_values, index=dates_fc, name='forecast')

    # Compile results for this series
    result_df = pd.DataFrame({
        Config.BRANCH_COL: branch,
        Config.ITEM_COL: item,
        'forecast': fc_series
    }, index=dates_fc)
    all_results.append(result_df)

    # Evaluate
    eval_df = pd.DataFrame({
        'test': test_series,
        'forecast': fc_series
    }).dropna()
    if not eval_df.empty:
        mae = mean_absolute_error(eval_df['test'], eval_df['forecast'])
        mape = mean_absolute_percentage_error(eval_df['test'], eval_df['forecast']) * 100
        evaluations.append((branch, item, mae, mape))

# 4. Save all forecasts
if all_results:
    combined = pd.concat(all_results)
    out_all = os.path.join(Config.OUTPUT_DIR, 'arima_forecasts_all.csv')
    combined.to_csv(out_all)
    print(f"Saved all forecasts to {out_all}")
else:
    print("No forecasts generated.")

# 5. Print evaluation summary
if evaluations:
    print("\nEvaluation Summary (MAE, MAPE%):")
    for branch, item, mae, mape in evaluations:
        print(f"{branch} | {item}: MAE={mae:.2f}, MAPE={mape:.2f}%")

# End of script


INFO:root:Forecast horizon: 30 days until 2024-04-30
/Users/ivorymykagalang/Documents/MSDS_2023/Capstone/Use Cases/04_Biggs/03_Modeling/modeling_ivy/.venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/ivorymykagalang/Documents/MSDS_2023/Capstone/Use Cases/04_Biggs/03_Modeling/modeling_ivy/.venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/ivorymykagalang/Documents/MSDS_2023/Capstone/Use Cases/04_Biggs/03_Modeling/modeling_ivy/.venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/ivorymykagalang/Documents/MSDS_2023/Capstone/Use Cases/04_Big

Saved all forecasts to results/arima_forecasts_all.csv

Evaluation Summary (MAE, MAPE%):
BRLN | BIGGS CAJUN CHICKEN 1PC: MAE=6.80, MAPE=38.14%
BRLN | BIGGS CHICKEN 1PC: MAE=41.18, MAPE=51.57%
BRLN | CHEESY BURGER: MAE=6.82, MAPE=50.40%
BRLN | GOTO W EGG: MAE=6.44, MAPE=40.91%
BRLN | ICED TEA: MAE=18.82, MAPE=44.34%
BRLN | PINEAPPLE JUICE: MAE=20.45, MAPE=93.33%
BRLN | SPAGHETTI: MAE=9.82, MAPE=48.80%
OLA | GOTO W EGG: MAE=41.18, MAPE=27.28%
OLA | PINEAPPLE JUICE: MAE=19.36, MAPE=22.59%


/Users/ivorymykagalang/Documents/MSDS_2023/Capstone/Use Cases/04_Biggs/03_Modeling/modeling_ivy/.venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/ivorymykagalang/Documents/MSDS_2023/Capstone/Use Cases/04_Biggs/03_Modeling/modeling_ivy/.venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/ivorymykagalang/Documents/MSDS_2023/Capstone/Use Cases/04_Biggs/03_Modeling/modeling_ivy/.venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._i